# Python으로 웹 스크래퍼 만들기 2

사이트 운영자들은 웹 스크래핑 매우 싫어한다.

그래서 봇 인증 프로그램 등을 만들어 작업을 까다롭게 조치를 해두는 경우가 있다. ( 이용약관에도 기입해 두기도 한다. )

민감한 사항이 때문에 스크래핑한 내용을 상업적으로 사용할 경우 법적다툼이 벌어질 가능성도 있다.

교육 목적 이외에 상업적인 서비스를 만들 생각이라면 참고하자.

---


- URL Formatting

- Requests

- [HTTP Status Codes](https://developer.mozilla.org/ko/docs/Web/HTTP/Status)

- dictionary

- .startswith

- f"{}"

- find_all : list

- find : v

---

## Refactoring

- search_term 을 parameter 로 받고, dict 데이터로 results 를 return 하는 함수 생성


In [2]:
from requests import get 
from bs4 import BeautifulSoup

# extract 폴더, wwr.py 파일, extract_jobs 함수 import

from extract.wwr import extract_jobs

jobs = extract_jobs('python')

print(jobs)

print(len(jobs))

[{'position': 'Full-Time', 'company': 'OpenCraft', 'location': 'Anywhere in the World', 'link': 'https://weworkremotely.com/remote-jobs/opencraft-senior-open-source-developer-devops-python-django-react-aws-openstack'}, {'position': 'Full-Time', 'company': 'Close', 'location': 'USA Only', 'link': 'https://weworkremotely.com/remote-jobs/close-senior-staff-software-engineer-backend-python-usa-100-remote'}, {'position': 'Full-Time', 'company': 'Proxify AB', 'location': 'Latin America Only/Europe Only/EMEA Only', 'link': 'https://weworkremotely.com/remote-jobs/proxify-ab-senior-backend-python-engineer-long-term-job-100-remote'}]
3


---

## Request 응답 거절

- Request로 접속, response code : 403


- 403 Forbidden

클라이언트는 콘텐츠에 접근할 권리를 가지고 있지 않습니다. 

예를들어 그들은 미승인이어서 서버는 거절을 위한 적절한 응답을 보냅니다. 401과 다른 점은 서버가 클라이언트가 누구인지 알고 있습니다.


In [5]:
from requests import get
from bs4 import BeautifulSoup

base_url = "https://kr.indeed.com/jobs?q="
search_term = "python"

response = get(f"{base_url}{search_term}")

if response.status_code != 200:
    print("Can't request website")
else:
    print(response.text)

Can't request website


In [6]:
print(response.status_code)

403


---

## Selenium

- https://www.selenium.dev/

- requests get 으로 403 forbidden, Selenium 을 이용하여 해결
---

BeautifulSoup 한계

바로, "자바스크립트로 동적으로 생성된 정보는 가져올 수 없다!"입니다.

자바스크립트가 발전을 하면서, Ajax(비동기 통신) 형태로 서버와 데이터를 주고 받아 화면에 뿌려주는 사이트가 많아 졌습니다. 

이러한 형식으로 데이터를 주고 받으면 url 변경이나 새로고침 없이 데이터를 가져오게 됩니다.

아마도 스크래핑을 시도하다가 데이터를 가져와야 정상인데, 아무것도 가져오지 못하는 현상을 마주친적이 있을 거에요. 

대부분의 경우가 자바스크립트로 HTML을 만들어서 그렇습니다.

Selenium 라이브러리를 사용하는 이유는 다음과 같습니다.

    1. 자바스크립트가 동적으로 만든 데이터를 크롤링 하기 위해

    2. 사이트의 다양한 HTML 요소에 클릭, 키보드 입력 등 이벤트를 주기 위해

Selenium을 잘 활용하면, 평소에 반복적으로 하고 있는 웹상의 업무를 자동화할 수도 있습니다.



---

In [7]:
# indeed 403 fix

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

# for Replit option

# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

# response = get(f"{base_url}{search_term}")

browser.get(f"{base_url}?q={search_term}")

print(browser.page_source) # html source code 추출


<html dir="ltr" lang="ko" class="js-focus-visible" data-js-focus-visible=""><head>
    <link rel="shortcut icon" href="/images/favicon.ico">
    <title>2023년 5월 24일 Python 취업, 일자리, 채용 | Indeed.com</title>
    <meta http-equiv="content-type" content="text/html; charset=utf-8">
    <meta name="description" content="[2023 Summer Internship] Medtech – Business Excellence, 관리 프로그램 개발자, 인턴 외에도 848 건 이상의 Python 관련 일자리가 Indeed.com에 있습니다!">
    <meta name="keywords" content="Python 취업, 채용정보,  careers,  employment,  job listings,  job search,  search engine, work in ">
    <meta name="referrer" content="origin-when-cross-origin">
    <link rel="canonical" href="https://kr.indeed.com/q-python-%ec%b1%84%ec%9a%a9%ea%b3%b5%ea%b3%a0.html">
    <link rel="alternate" href="android-app://com.indeed.android.jobsearch/https/kr.indeed.com/m/jobs?q=python">
    <link rel="alternate" href="ios-app://https/kr.indeed.com/m/jobs?q=python">
    <link rel="alternate" type="application/rss+xml" title="Python 취업, 채

In [8]:
type(browser)

selenium.webdriver.chrome.webdriver.WebDriver

---

<img src = '2.png' height ='500' width = '500'>



## Recursive


1. find 로 `ul` 찾음


2. find_all 로 `ul` 안에 li 모두 찾음 (li 안에 li) 


- `ul` 바로 안에 `li` 만을 찾길 원함. 
-  `li` 안에 `li` 제외 
- 1 level 만 들어감 깊게 들어가는거 방지

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

browser.get(f"{base_url}?q={search_term}")

# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

job_list = soup.find("ul", class_="jobsearch-ResultsList")

jobs = job_list.find_all('li', recursive=False)

# ul 태그 내부, li 태그 (1 level)의 개수 출력
print(len(jobs))
print()

# jobs 리스트, li 태그 데이터 출력
for job in jobs:
    print(job)
    print()

18

<li><div class="cardOutline tapItem fs-unmask result job_ed7b7e6eb72ffab8 resultWithShelf sponTapItem desktop vjs-highlight css-kyg8or eu4oa1w0"><div class="slider_container css-77eoo7 eu4oa1w0" dir="auto"><div class="slider_list css-kyg8or eu4oa1w0"><div class="slider_item css-kyg8or eu4oa1w0"><div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="jobCard_mainContent big6_visualChanges" role="presentation"><tbody><tr><td class="resultContent"><div class="css-1m4cuuf e37uo190"><h2 class="jobTitle css-1h4a4n5 eu4oa1w0" tabindex="-1"><a aria-label="2023 Software Engineering Intern, People with Disabilities (장애인 채용)의 전체 세부 정보" class="jcs-JobTitle css-jspxzf eu4oa1w0" data-hide-spinner="true" data-hiring-event="false" data-jk="ed7b7e6eb72ffab8" data-mobtk="1h167dravknto804" href="/rc/clk?jk=ed7b7e6eb72ffab8&amp;fccid=a5b4499d9e91a5c6&amp;vjs=3" id="job_ed7b7e6eb72ffab8" role="button" target="_blank"><span id="jobTitle-ed7b7e6eb72ffab8" title="2023 Software Engineeri

---

## 데이터 필터링

- jobs 리스트 중 

###  None

- 무언가 '없읍'

- false 랑 다름

- html 데이터 -> ul -> li (recursive=False)

- 이 후 필터링 으로 "div", class_="mosaic-zone" 데이터 처리 



<img src = '3.png' height ='500' width = '500'>

In [14]:
# li 태그 중 class 네임에 따라 출력, # find, none 논리로 출력 

for job in jobs:
    
    zone = job.find("div", class_="mosaic-zone")
    if zone == None:
        print("job li")
    else:
        print("mosaic li")

job li
job li
job li
job li
job li
mosaic li
job li
job li
job li
job li
job li
mosaic li
job li
job li
job li
job li
job li
mosaic li


---

In [15]:
# class name으로 데이터 필터링

for job in jobs:
        
    if job.find("div", class_="cardOutline"):
        print("job li")
    else:
        print("mosaic li")

job li
job li
job li
job li
job li
mosaic li
job li
job li
job li
job li
job li
mosaic li
job li
job li
job li
job li
job li
mosaic li


---

## Select

- select_one > dict 형으로 한번에 

- find_all('') > 태그 를 리스트화 


- https://nomadcoders.co/python-for-beginners/lectures/3801

In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://kr.indeed.com/jobs"
search_term = "python"

browser.get(f"{base_url}?q={search_term}")

# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

job_list = soup.find("ul", class_="jobsearch-ResultsList")

jobs = job_list.find_all('li', recursive=False)


# li 태그 중 class 네임에 따라 출력,
# find, none 논리로 출력 

results = []

for job in jobs:
    
    zone = job.find("div", class_="mosaic-zone")
    
    if zone == None:
        
        # anchor = job.find_all('a') # list

        anchor = job.select_one('h2 a') # dict
        
        title = anchor['aria-label']
        link = anchor['href']
        
        company = job.find('span', class_='companyName')
        location = job.find('div', class_='companyLocation')
    
        # dict 데이터 형
        job_data = {
            'position': title,
            'link': f"https://kr.indeed.com{link}",
            'company': company.string,
            'location': location.string
        }
        
        # 결과 리스트 append
        results.append(job_data)

    
for result in results:
    print(result)
    print()

{'position': '2023 Software Engineering Intern, People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=ed7b7e6eb72ffab8&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '2023 Student Training in Engineering Program (STEP), People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5d24a062cabc9f0d&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '[송파구] 데이터분석(Python,R,모델,ML)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=a0f0a1cb28e20f8c&fccid=59060ddcdf96b101&vjs=3', 'company': '시니어앤파트너즈', 'location': '서울 송파구'}

{'position': '인공지능 관리 및 개발자 (Python, yolo 사용자 우대)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b469f992ddab8f71&fccid=aa11023cc66bb11b&vjs=3', 'company': '하가', 'location': '서울 동작구'}

{'position': 'IT분야 직업훈련교사의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5594832db046013e&fccid=af1182589e05cf0b&vjs=3', 'company': '재단법인 한국아이티교육재단',

In [17]:
print(title)

백엔드 엔지니어 (Python) 정규직의 전체 세부 정보


In [18]:
print(link)

/rc/clk?jk=79d696388d32d3ff&fccid=dd6f5fd8e8c02e7e&vjs=3


In [22]:
print(company)

<span class="companyName">데이터비</span>


In [20]:
print(location)

<div class="companyLocation">서울 마포구</div>


---

## pages


- 페이지가 1개 

- 페이지가 여러개, 이동 버튼 없음

- 페이지가 여러개, 이동 버튼 있음 (6개이상 존재)


1. 태그랑, class 네임 둘다 변경 `nav`, `div`, `css-jbuxu0 ecydgvn0`


2. `pagination = soup.find('nav', class_='css-jbuxu0 ecydgvn0')`   시 일단 모두 생성됨. none 으로 return x 일단 생성 o


3. `pages = pagination.find_all('div', recursive=False)` 있으면 리스트화, 없으면 [] 빈 리스트 
    - <class 'bs4.element.ResultSet'> 
    
 



In [34]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def get_page_count(keyword):
    
    options = Options()
    browser = webdriver.Chrome(options=options)

    base_url = "https://kr.indeed.com/jobs"
    browser.get(f"{base_url}?q={keyword}")
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    pagination = soup.find('nav', class_='css-jbuxu0 ecydgvn0')    
    
    pages = pagination.find_all('div', recursive=False)
    
    length = len(pages)
    
    # 5를 초과하는 6 이상 부터는 처리 하지 않을 예정
    if length == 0:
        return 1
    elif length > 5:
        return 5
    else: 
        return length

In [35]:
get_page_count('react')


5

In [36]:
get_page_count('nest')


1

In [37]:
get_page_count('python')


5

## Refactoring - 2

- 함수화
- pages 추가


In [38]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def extract_indeed_jobs(keyword):
    
    pages = get_page_count(keyword)
    
    for page in range(pages):
        
        options = Options()

        browser = webdriver.Chrome(options=options)
        
        base_url = "https://kr.indeed.com/jobs"
        search_term = f'{keyword}'

        browser.get(f"{base_url}?q={search_term}")

        # --- bs4 처리 

        soup = BeautifulSoup(browser.page_source, "html.parser")

        job_list = soup.find("ul", class_="jobsearch-ResultsList")

        jobs = job_list.find_all('li', recursive=False)


        # li 태그 중 class 네임에 따라 출력,
        # find, none 논리로 출력 

        results = []

        for job in jobs:

            zone = job.find("div", class_="mosaic-zone")

            if zone == None:

                # anchor = job.find_all('a') # list

                anchor = job.select_one('h2 a') # dict

                title = anchor['aria-label']
                link = anchor['href']

                company = job.find('span', class_='companyName')
                location = job.find('div', class_='companyLocation')

                # dict 데이터 형
                job_data = {
                    'position': title,
                    'link': f"https://kr.indeed.com{link}",
                    'company': company.string,
                    'location': location.string
                }

                # 결과 리스트 append
                results.append(job_data)

        for result in results:
            print(result)
            print()

In [39]:
# 1번 페이지 5번 반복 

extract_indeed_jobs('python')

{'position': '2023 Software Engineering Intern, People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=ed7b7e6eb72ffab8&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '2023 Student Training in Engineering Program (STEP), People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5d24a062cabc9f0d&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '[송파구] 데이터분석(Python,R,모델,ML)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=a0f0a1cb28e20f8c&fccid=59060ddcdf96b101&vjs=3', 'company': '시니어앤파트너즈', 'location': '서울 송파구'}

{'position': '인공지능 관리 및 개발자 (Python, yolo 사용자 우대)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b469f992ddab8f71&fccid=aa11023cc66bb11b&vjs=3', 'company': '하가', 'location': '서울 동작구'}

{'position': 'IT분야 직업훈련교사의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5594832db046013e&fccid=af1182589e05cf0b&vjs=3', 'company': '재단법인 한국아이티교육재단',

{'position': '2023 Software Engineering Intern, People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=ed7b7e6eb72ffab8&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '2023 Student Training in Engineering Program (STEP), People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5d24a062cabc9f0d&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '[송파구] 데이터분석(Python,R,모델,ML)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=a0f0a1cb28e20f8c&fccid=59060ddcdf96b101&vjs=3', 'company': '시니어앤파트너즈', 'location': '서울 송파구'}

{'position': '인공지능 관리 및 개발자 (Python, yolo 사용자 우대)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b469f992ddab8f71&fccid=aa11023cc66bb11b&vjs=3', 'company': '하가', 'location': '서울 동작구'}

{'position': 'IT분야 직업훈련교사의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5594832db046013e&fccid=af1182589e05cf0b&vjs=3', 'company': '재단법인 한국아이티교육재단',

---

`options = Options()`
`browser = webdriver.Chrome(options=options)`

를 함수 내에서 실행시키게 해야함.

- 켜져있는 상태로, 한번 더 켜지면, 봇 의심

- 함수 내에서 오픈하게 되면, 실행 후, 꺼지고, 다시 실행

- 오픈, 카운트, 클로즈

- 1~5. 오픈, 출력, 클로즈



---

## Refactoring -3

- 1 ~ 5 번페이지 start 처리

- page 에 따라 start 부분 처리 x(0), 10, 20

- start를 0처리 하면, start 없이 요청 하는것과 같은 결과가 나옴

- result 빼서 처리

In [40]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def extract_indeed_jobs(keyword):
    
    pages = get_page_count(keyword)
    
    results = []
  
    for page in range(pages):
        
        options = Options()

        browser = webdriver.Chrome(options=options)
        
        base_url = "https://kr.indeed.com/jobs"
        search_term = f'{keyword}'

        browser.get(f"{base_url}?q={search_term}&start={page*10}")

        # --- bs4 처리 

        soup = BeautifulSoup(browser.page_source, "html.parser")

        job_list = soup.find("ul", class_="jobsearch-ResultsList")

        jobs = job_list.find_all('li', recursive=False)


        # li 태그 중 class 네임에 따라 출력,
        # find, none 논리로 출력 


        for job in jobs:

            zone = job.find("div", class_="mosaic-zone")

            if zone == None:

                # anchor = job.find_all('a') # list

                anchor = job.select_one('h2 a') # dict

                title = anchor['aria-label']
                link = anchor['href']

                company = job.find('span', class_='companyName')
                location = job.find('div', class_='companyLocation')

                # dict 데이터 형
                job_data = {
                    'position': title,
                    'link': f"https://kr.indeed.com{link}",
                    'company': company.string,
                    'location': location.string
                }

                # 결과 리스트 append
                results.append(job_data)

    
    for result in results:
        print(result)
        print()

In [41]:
extract_indeed_jobs('python')

{'position': '2023 Software Engineering Intern, People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=ed7b7e6eb72ffab8&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '2023 Student Training in Engineering Program (STEP), People with Disabilities (장애인 채용)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5d24a062cabc9f0d&fccid=a5b4499d9e91a5c6&vjs=3', 'company': 'Google', 'location': '서울'}

{'position': '[송파구] 데이터분석(Python,R,모델,ML)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=a0f0a1cb28e20f8c&fccid=59060ddcdf96b101&vjs=3', 'company': '시니어앤파트너즈', 'location': '서울 송파구'}

{'position': '인공지능 관리 및 개발자 (Python, yolo 사용자 우대)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=b469f992ddab8f71&fccid=aa11023cc66bb11b&vjs=3', 'company': '하가', 'location': '서울 동작구'}

{'position': 'IT분야 직업훈련교사의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=5594832db046013e&fccid=af1182589e05cf0b&vjs=3', 'company': '재단법인 한국아이티교육재단',

---

- 완전 함수로 변환

- return 리스트

In [42]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def extract_indeed_jobs(keyword):
    
    pages = get_page_count(keyword)
    
    results = []
  
    for page in range(pages):
        
        options = Options()

        browser = webdriver.Chrome(options=options)
        
        base_url = "https://kr.indeed.com/jobs"
        search_term = f'{keyword}'
        final_url = f"{base_url}?q={search_term}&start={page*10}"
        
        print(f'Request url : {final_url}')
        browser.get(final_url)

        # --- bs4 처리 

        soup = BeautifulSoup(browser.page_source, "html.parser")

        job_list = soup.find("ul", class_="jobsearch-ResultsList")

        jobs = job_list.find_all('li', recursive=False)


        # li 태그 중 class 네임에 따라 출력,
        # find, none 논리로 출력 


        for job in jobs:

            zone = job.find("div", class_="mosaic-zone")

            if zone == None:

                # anchor = job.find_all('a') # list

                anchor = job.select_one('h2 a') # dict

                title = anchor['aria-label']
                link = anchor['href']

                company = job.find('span', class_='companyName')
                location = job.find('div', class_='companyLocation')

                # dict 데이터 형
                job_data = {
                    'position': title,
                    'link': f"https://kr.indeed.com{link}",
                    'company': company.string,
                    'location': location.string
                }

                # 결과 리스트 append
                results.append(job_data)

    
    return results

In [47]:
# dict 데이터 형 job_data를 담고 있는 리스트 데이터 jobs


jobs = extract_indeed_jobs('python')

print(jobs) 

Request url : https://kr.indeed.com/jobs?q=nest&start=0
[{'position': '모빌리티엔 자동차보험 웹서비스 백엔드 개발자의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=f7920ae82f186b88&fccid=77914dd9da8d71f8&vjs=3', 'company': '모빌리티엔', 'location': '서울 성동구'}, {'position': 'Partner Manager, APAC의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=2e4aaa8f357f10c1&fccid=9ba9500ecf2596c5&vjs=3', 'company': 'Sendbird', 'location': '서울'}, {'position': '웹 서비스 및 모바일 앱의 UI 디자이너의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=ab91288ca5716c8b&fccid=550550f565ca2d67&vjs=3', 'company': '브레싱스 주식회사 관심기업', 'location': '서울 역삼동'}, {'position': 'CAD/ CAM 설계원 (신입 및 경력자 별도협의)의 전체 세부 정보', 'link': 'https://kr.indeed.com/rc/clk?jk=776425242e3dbbc9&fccid=6fcc48d21fceda12&vjs=3', 'company': '에스에이치엘', 'location': '대구 달성군'}]


In [48]:
print(len(jobs))

4


- 첫번째 페이지로 이동후, 그 이후 페이지가 몇개 있는지 확인, 제약사항(5) 후 처리
